In [ ]:
!pip install river
%pip install sdv

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:

from river.datasets import Phishing
import pandas as pd
import numpy as np
from utils import plot_drift
from novelty import ephemeral, persistant, periodic, degenerate

from river import datasets
from river import ensemble
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing


from drift import detect_drift
from river.drift import KSWIN
from river import stream, metrics, evaluate, ensemble, linear_model, preprocessing
import copy
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
phishing = Phishing()
list(phishing.take(2))

[({'empty_server_form_handler': 0.0,
   'popup_window': 0.0,
   'https': 0.0,
   'request_from_other_domain': 0.0,
   'anchor_from_other_domain': 0.0,
   'is_popular': 0.5,
   'long_url': 1.0,
   'age_of_domain': 1,
   'ip_in_url': 1},
  True),
 ({'empty_server_form_handler': 1.0,
   'popup_window': 0.0,
   'https': 0.5,
   'request_from_other_domain': 0.5,
   'anchor_from_other_domain': 0.0,
   'is_popular': 0.5,
   'long_url': 0.0,
   'age_of_domain': 1,
   'ip_in_url': 0},
  True)]

In [ ]:
def dataset_to_df(dataset, count=None):
    if not count:
        count = len(list(dataset))
    data_list = [{**x, 'value': y} for x, y in dataset.take(count)]
    return pd.DataFrame(data_list)

def df_to_dataset(df):
    df_nonvalue = df.loc[:, df.columns != 'value']
    df_value = df['value']
    return list(zip(df_nonvalue.to_dict(orient='records'), df_value.to_list()))

In [ ]:
df = dataset_to_df(phishing)
df

,empty_server_form_handler,popup_window,https,request_from_other_domain,anchor_from_other_domain,is_popular,long_url,age_of_domain,ip_in_url,value
0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1,1,True
1,1.0,0.0,0.5,0.5,0.0,0.5,0.0,1,0,True
2,0.0,0.0,1.0,0.0,0.5,0.5,0.0,1,0,True
3,0.0,0.0,1.0,0.0,0.0,1.0,0.5,0,0,True
4,1.0,0.0,0.5,1.0,0.0,0.5,0.5,1,0,False
...,...,...,...,...,...,...,...,...,...,...
1245,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1,0,True
1246,0.0,0.5,1.0,0.5,0.0,0.5,0.5,1,0,False
1247,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0,0,True
1248,0.5,0.5,1.0,0.5,0.5,0.5,0.0,1,0,True


In [ ]:
def eval_with_kswin(dataset):
    metric = metrics.F1()

    base_model = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    models = [copy.deepcopy(base_model) for _ in range(10)]
    detectors = [KSWIN(alpha=0.005, window_size=100, stat_size=30) for _ in range(10)]

    y_true_all = []
    y_pred_all = []

    for x, y in dataset:
        model_preds = []
        for i, (model, detector) in enumerate(zip(models, detectors)):
            y_pred = model.predict_one(x)
            model_preds.append(y_pred)

            # Feed one feature to KSWIN (choose one or loop over features)
            detector.update(x[list(x)[0]])  # Simple: first feature only
            is_drift = detector.drift_detected
            if is_drift:
                models[i] = copy.deepcopy(base_model)  # Reset model on drift
                detectors[i] = KSWIN(alpha=0.005, window_size=100, stat_size=30)

        # Majority vote (simple)
        y_pred_ensemble = max(set(model_preds), key=model_preds.count)

        y_true_all.append(y)
        y_pred_all.append(y_pred_ensemble)

        # Update all models
        for model in models:
            model.learn_one(x, y)

    f1 = metrics.F1()
    for yt, yp in zip(y_true_all, y_pred_all):
        f1.update(yt, yp)

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )
    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"KSWIN: {f1}")
    print(f"plain: {plain_result}")

Ephemeral Novelty

In [34]:
print("Pre-Novelty Insertion")
eval_with_kswin(phishing)

print()
print("Post-Novelty Insertion")
df_ephemeral = ephemeral(df, 'value', 200, 500, lambda x: not x)
eval_with_kswin(df_to_dataset(df_ephemeral))

Pre-Novelty Insertion
KSWIN: F1: 87.97%
plain: F1: 87.97%

Post-Novelty Insertion
KSWIN: F1: 75.88%
plain: F1: 75.88%


In [35]:
def eval_Ada(dataset):
    model_Ada = ensemble.AdaBoostClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=5,

    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    metric = metrics.F1()

    Ada_result = evaluate.progressive_val_score(
        dataset,
        model_Ada,
        metric,)

    print(f"Ada: {Ada_result}")


    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

In [36]:
print("Pre-Novelty")
eval_Ada(phishing)

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_ephemeral))

Pre-Novelty
Ada: F1: 87.43%
plain: F1: 87.70%

Post-Novelty Insertion
Ada: F1: 87.48%
plain: F1: 81.56%


Persistant Novelty

In [37]:
print("Pre-Novelty Insertion")
eval_with_kswin(phishing)

print()
print("Post-Novelty Insertion")
df_persistant = persistant(df, 'value', 600, lambda x: not x) # insert persistant novelty
eval_with_kswin(df_to_dataset(df_persistant))

Pre-Novelty Insertion
KSWIN: F1: 87.97%
plain: F1: 87.97%

Post-Novelty Insertion
KSWIN: F1: 76.12%
plain: F1: 76.12%


In [38]:
print("Pre-Novelty")
eval_Ada(phishing)

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_persistant))

Pre-Novelty
Ada: F1: 87.50%
plain: F1: 87.74%

Post-Novelty Insertion
Ada: F1: 86.06%
plain: F1: 81.16%


Periodic Novelty

In [39]:
print("Pre-Novelty Insertion")
eval_with_kswin(phishing)

print()
print("Post-Novelty Insertion")
df_periodic = periodic(df, 'value', location=200, duration=10, period=100, func=lambda x: not x) # insert periodic

eval_with_kswin(df_to_dataset(df_periodic))

Pre-Novelty Insertion
KSWIN: F1: 87.97%
plain: F1: 87.97%

Post-Novelty Insertion
KSWIN: F1: 79.82%
plain: F1: 79.82%


In [40]:
print("Pre-Novelty")
eval_Ada(phishing)

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_periodic))

Pre-Novelty
Ada: F1: 87.13%
plain: F1: 87.56%

Post-Novelty Insertion
Ada: F1: 77.24%
plain: F1: 78.52%


Degenerate Novelty

In [42]:
print("Pre-Novelty Insertion")
eval_with_kswin(phishing)

print()
print("Post-Novelty Insertion")
df_degenerate = degenerate(df, 'value', start=300, duration=1000, func=lambda x: x + np.random.randint(-1000, 1000))

eval_with_kswin(df_to_dataset(df_degenerate))

Pre-Novelty Insertion
KSWIN: F1: 87.87%
plain: F1: 87.97%

Post-Novelty Insertion


/content/novelty.py:80: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[  27  272  -68  353  270  359  207 -313  318 -471  349  -60 -464  550
 -340  912 -171 -823  -97 -420 -267 -359  872 -868  676 -174  828 -716
 -300 -580  747  667  733   41 -767 -178 -566 -406 -180 -484  189 -606
 -161 -802 -311 -165 -757 -465 -666  325 -609  961  610  657   98  398
  145  470 -235 -730 -300  414  706 -711  676  211  338  776 -370 -145
  -76 -898 -950 -374  596  198  117 -225 -215   89  387 -236  117 -118
  329 -619 -454 -118  458 -391   24 -243 -162 -448  287  -79 -820  640
   18  725  333  332  443   18 -762  876  209  518 -339 -321  192  -65
 -155  462 -140  756 -122 -298  779  910 -373  138  448  470 -572 -152
  623  642   66  691  252  782  802  149 -949  952 -798  427 -896  398
 -611   15  528 -338 -992  848 -680 -107 -882  672 -185  488  879  491
  700 -340 -294  865  842 -211  115 -260 -721 -881 -809  544  530 

KSWIN: F1: 29.05%
plain: F1: 29.05%


In [43]:
print("Pre-Novelty")
eval_Ada(phishing)

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_degenerate))

Pre-Novelty
Ada: F1: 86.39%
plain: F1: 87.19%

Post-Novelty Insertion
Ada: F1: 28.10%
plain: F1: 28.57%
